# Analysis of DSC peaks using Spink method based on Freire and Biltonen deconvolution:

Come prima cosa è necessario caricare i dati solamente dei picchi: "ci_baseline.csv" dentro la cartella "ci_puliti".


In [1]:
# importo dati e librerie
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, Dropdown
import os

folder_path = r"/Users/marco/Desktop/TESI/LOCAL_P/c-MYC/DSC/ci_puliti"

# Inizializza dizionario
ci_finali_data = {}

# Caricamento
for filename in os.listdir(folder_path):
    if filename.endswith(".csv") and filename.startswith("c"):
        path = os.path.join(folder_path, filename)
        df = pd.read_csv(path, sep="\t")
        ci_finali_data[filename.replace(".csv", "")] = df

# Mostra le anteprime (primi 5 file)
for nome, df in list(ci_finali_data.items())[:5]:
    print(f"\n--- {nome} ---")
    display(df.head())


--- c3_cubica ---


,Temperatura,Cp_corretto,Cp_baseline,Cp_finale_cubica
0,61.746,-1959.506367,-1944.825828,-14.680539
1,61.788,-1958.516555,-1943.905216,-14.611339
2,61.829,-1947.603221,-1942.995848,-4.607373
3,61.871,-1946.613409,-1942.053398,-4.560011
4,61.913,-1945.745113,-1941.099952,-4.645161



--- c3_gradino ---


,Temperatura,Cp_corretto,Cp_baseline,Cp_finale_gradino
0,65.924,-1800.862102,-1584.702269,-216.159832
1,65.966,-1800.004606,-1584.702269,-215.302336
2,66.008,-1799.147110,-1584.702269,-214.444841
3,66.049,-1797.251830,-1584.702269,-212.549561
4,66.091,-1787.505445,-1584.702269,-202.803176



--- c3_lineare ---


,Temperatura,Cp_corretto,Cp_baseline,Cp_finale_lineare
0,65.924,-1800.862102,-1688.411241,-112.450860
1,65.966,-1800.004606,-1687.427917,-112.576689
2,66.008,-1799.147110,-1686.444593,-112.702517
3,66.049,-1797.251830,-1685.484681,-111.767149
4,66.091,-1787.505445,-1684.501357,-103.004088



--- c3_sigmoide ---


,Temperatura,Cp_corretto,Cp_baseline,Cp_finale_sigmoide
0,69.268,-1679.059537,-1496.248928,-182.810610
1,69.309,-1678.311278,-1496.229809,-182.081469
2,69.351,-1685.908819,-1496.209980,-189.698839
3,69.393,-1684.617471,-1496.189901,-188.427569
4,69.434,-1691.187557,-1496.170057,-195.017500



--- c5_cubica ---


,Temperatura,Cp_corretto,Cp_baseline,Cp_finale_cubica
0,66.293,-1265.469762,-1179.645830,-85.823931
1,66.335,-1255.187526,-1179.973553,-75.213973
2,66.376,-1252.769732,-1180.264900,-72.504832
3,66.418,-1251.376978,-1180.534186,-70.842792
4,66.460,-1241.095335,-1180.774061,-60.321274


## Preprocessing e costruzione del dataframe:

In [2]:
def plot_dsc_interattivo(dati_dict, colonna_cp_prefix="Cp_finale"):
    """
    Visualizza interattivamente i dati DSC con Cp finale e visualizzazione del picco.
    
    Parameters:
    - dati_dict: dizionario tipo {"c3_cubica": DataFrame, ...}
    - colonna_cp_prefix: prefisso della colonna Cp finale, default = 'Cp_finale'
    """
    opzioni = sorted(dati_dict.keys())

    @interact(dataset=Dropdown(options=opzioni, description='Campione:'))
    def _plot(dataset):
        df = dati_dict[dataset]
        baseline = dataset.split("_")[-1]
        col_cp = f'{colonna_cp_prefix}_{baseline}'
        
        T = df['Temperatura']
        Cp = df[col_cp]

        plt.figure(figsize=(8, 5))
        plt.plot(T, Cp, label=f'{dataset}', color='darkred')
        plt.title(f'DSC - {dataset}')
        plt.xlabel('Temperatura (°C)')
        plt.ylabel('Heat Capacity (cal/mol·°C)')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()


plot_dsc_interattivo(ci_finali_data)


interactive(children=(Dropdown(description='Campione:', options=('c3_cubica', 'c3_gradino', 'c3_lineare', 'c3_…

Si basa sullo studio di processi di unfolding con stati intermedi studiati da Spink utilizzando la convoluzione di Freire–Biltonen.
Essa si basa sulle seguenti equazioni:

**1. Schema della transizione**

Il modello considera una sequenza di stati interconnessi:

$$
I_0 \xrightarrow{K_1} I_1 \xrightarrow{K_2} I_2 \xrightarrow{K_3} \dots \xrightarrow{K_n} I_n
$$

Dove $I_0$ è lo stato iniziale (nativo), $I_n$ quello finale (completamente unfolded) e gli altri sono **intermedi**.

---

**2. Costanti di equilibrio**

Ogni equilibrio ha una costante associata:

$$
K_i = e^{-\Delta G_i / RT}, \quad \text{con} \quad \Delta G_i = G_i - G_0
$$

$K_i$ dà informazioni su quanto sia probabile il passaggio dallo stato nativo allo stato intermedio i-esimo; un valore molto grande indica una grande stabilità dello stato intermedio.

---

**3. Funzione di partizione totale**

La funzione di partizione tiene conto di tutti gli stati:

$$
Q = 1 + \sum_{i=1}^{n-1} e^{-\Delta G_i / RT} + e^{-\Delta G_n / RT} 
$$

Rappresenta la somma pesata di tutti gli stati accessibili a una certa temperatura. Interpretabile come l'indice di probabilità che uno stato sia popolato ad una determinata T ed è essenziale (insieme a $K_i$) al fine di trovare la frazione di stati popolati $F_i$.

---

**4. Frazione della popolazione in ogni stato**

Per lo stato $i$:

$$
F_i = \frac{e^{-\Delta G_i / RT}}{Q} 
$$

In particolare:

- Stato iniziale:  
  $$
  F_0 = \frac{1}{Q} 
  $$

- Stato finale (solo due stati):
  $$
  F_n = \frac{K_n}{1 + K_n}
  $$

---

**5. Collegamento con i dati calorimetrici**

La funzione di partizione può essere ottenuta direttamente dalla **entalpia cumulativa media** $⟨\Delta H⟩(T)$:

$$
Q(T) = \exp \left( \int_{T_0}^{T} \frac{\langle \Delta H \rangle}{RT^2} \, dT \right)
$$

dove:

$$
\langle \Delta H \rangle = \int_{T_0}^{T} \left[ C_p(\text{ex}) - C_p(\text{baseline}) \right] \, dT 
$$

---

**6. Legame tra entalpia e frazioni molecolari**

L’entalpia totale misurata è somma pesata:

$$
\langle \Delta H \rangle(T) = \sum_{i=0}^n \Delta H_i \cdot F_i(T)
$$

---

**7. Frazione nello stato finale**

Si può anche calcolare direttamente:

$$
F_n = \exp \left( - \int_{T}^{T_n} \frac{\Delta H_n - \langle \Delta H \rangle}{RT^2} \, dT \right) 
$$

---

**8. Energia libera di ogni stato**

Ogni stato è caratterizzato da un’energia libera:

$$
\Delta G_i(T) = \Delta H_i + \Delta C_{p,i}(T - T_0) - T \Delta S_i + \Delta C_{p,i} \ln \left( \frac{T}{T_0} \right) T
$$


In [3]:
def process_calorimetric_data(df, col_temp='Temperatura', col_cp='Cp_finale_cubica'):
    """
    Applica la deconvoluzione Freire-Biltonen a un DataFrame contenente i dati DSC corretti.
    """
    R = 1.987  # cal/(mol·K)

    T = df[col_temp].values + 273.15  # Converti da °C a K
    Cp = df[col_cp].values            # Usa Cp già in cal/(mol·°C)

    df = df.copy()
    df['T_K'] = T
    df['Cp'] = Cp

    # 1. Entalpia cumulativa ⟨ΔH⟩(T)
    h_dhi = np.cumsum(Cp * np.diff(T, prepend=T[0]))
    df['hDHi'] = h_dhi

    # 2. hDHi / RT²
    df['hDHi_RT2'] = h_dhi / (R * T**2)

    # 3. ∫(hDHi/RT²) dT
    df['∫(hDHi/RT²)dT'] = np.cumsum(df['hDHi_RT2'] * np.diff(T, prepend=T[0]))

    # 4. Q(T)
    df['Q'] = np.exp(df['∫(hDHi/RT²)dT'])

    # 5. Frazione nativa
    df['F0'] = 1 / df['Q']
    df['F0'] = np.clip(df['F0'], 0, 1)

    # 6. ΔHn - hDHi
    delta_Hn = h_dhi[-1]
    df['ΔHn - hDHi'] = delta_Hn - h_dhi

    # 7. (ΔHn - hDHi) / RT²
    df['(ΔHn-hDHi)/RT²'] = df['ΔHn - hDHi'] / (R * T**2)

    # 8. ∫[(ΔHn-hDHi)/RT²] dT
    df['∫(ΔHn-hDHi)/RT² dT'] = np.cumsum(df['(ΔHn-hDHi)/RT²'] * np.diff(T, prepend=T[0]))

    # 9. [Max - ∫]
    max_val = df['∫(ΔHn-hDHi)/RT² dT'].max()
    df['[Max - ∫]'] = max_val - df['∫(ΔHn-hDHi)/RT² dT']

    # 10. Fn
    df['Fn'] = np.exp(-df['[Max - ∫]'])
    df['Fn'] = np.clip(df['Fn'], 0, 1)


    # 11. Fi
    df['Fi'] = 1 - df['F0'] - df['Fn']
    df['Fi'] = np.clip(df['Fi'], 0, 1)

    return df



df_spink = {}
for nome, df in ci_finali_data.items():
    baseline = nome.split('_')[-1]
    col_cp = f'Cp_finale_{baseline}'
    df_spink[nome] = process_calorimetric_data(df, col_temp='Temperatura', col_cp=col_cp)


# Mostra le anteprime dei primi 3 DataFrame processati
for nome, df in list(df_spink.items())[:3]:
    print(f"\n--- {nome} ---")
    display(df.head())



--- c3_cubica ---


,Temperatura,Cp_corretto,Cp_baseline,Cp_finale_cubica,T_K,Cp,hDHi,hDHi_RT2,∫(hDHi/RT²)dT,Q,F0,ΔHn - hDHi,(ΔHn-hDHi)/RT²,∫(ΔHn-hDHi)/RT² dT,[Max - ∫],Fn,Fi
0,61.746,-1959.506367,-1944.825828,-14.680539,334.896,-14.680539,-0.000000,-0.000000,-0.000000e+00,1.000000,1.0,30698.373717,0.137752,0.000000,3.347839,0.035160,0.0
1,61.788,-1958.516555,-1943.905216,-14.611339,334.938,-14.611339,-0.613676,-0.000003,-1.156277e-07,1.000000,1.0,30698.987394,0.137720,0.005784,3.342055,0.035364,0.0
2,61.829,-1947.603221,-1942.995848,-4.607373,334.979,-4.607373,-0.802579,-0.000004,-2.632114e-07,1.000000,1.0,30699.176296,0.137687,0.011429,3.336410,0.035564,0.0
3,61.871,-1946.613409,-1942.053398,-4.560011,335.021,-4.560011,-0.994099,-0.000004,-4.504248e-07,1.000000,1.0,30699.367816,0.137654,0.017211,3.330628,0.035771,0.0
4,61.913,-1945.745113,-1941.099952,-4.645161,335.063,-4.645161,-1.189196,-0.000005,-6.743237e-07,0.999999,1.0,30699.562913,0.137620,0.022991,3.324848,0.035978,0.0



--- c3_gradino ---


,Temperatura,Cp_corretto,Cp_baseline,Cp_finale_gradino,T_K,Cp,hDHi,hDHi_RT2,∫(hDHi/RT²)dT,Q,F0,ΔHn - hDHi,(ΔHn-hDHi)/RT²,∫(ΔHn-hDHi)/RT² dT,[Max - ∫],Fn,Fi
0,65.924,-1800.862102,-1584.702269,-216.159832,339.074,-216.159832,-0.000000,-0.000000,-0.000000,1.000000,1.0,42789.060249,0.187304,0.000000,3.747996,0.023565,0.0
1,65.966,-1800.004606,-1584.702269,-215.302336,339.116,-215.302336,-9.042698,-0.000040,-0.000002,0.999998,1.0,42798.102948,0.187297,0.007866,3.740129,0.023751,0.0
2,66.008,-1799.147110,-1584.702269,-214.444841,339.158,-214.444841,-18.049381,-0.000079,-0.000005,0.999995,1.0,42807.109631,0.187290,0.015733,3.732263,0.023939,0.0
3,66.049,-1797.251830,-1584.702269,-212.549561,339.199,-212.549561,-26.763913,-0.000117,-0.000010,0.999990,1.0,42815.824163,0.187283,0.023411,3.724585,0.024123,0.0
4,66.091,-1787.505445,-1584.702269,-202.803176,339.241,-202.803176,-35.281647,-0.000154,-0.000016,0.999984,1.0,42824.341896,0.187273,0.031277,3.716719,0.024314,0.0



--- c3_lineare ---


,Temperatura,Cp_corretto,Cp_baseline,Cp_finale_lineare,T_K,Cp,hDHi,hDHi_RT2,∫(hDHi/RT²)dT,Q,F0,ΔHn - hDHi,(ΔHn-hDHi)/RT²,∫(ΔHn-hDHi)/RT² dT,[Max - ∫],Fn,Fi
0,65.924,-1800.862102,-1688.411241,-112.450860,339.074,-112.450860,-0.000000,-0.000000,-0.000000e+00,1.000000,1.0,39494.495884,0.172882,0.000000,3.512368,0.029826,0.0
1,65.966,-1800.004606,-1687.427917,-112.576689,339.116,-112.576689,-4.728221,-0.000021,-8.690657e-07,0.999999,1.0,39499.224105,0.172860,0.007260,3.505108,0.030044,0.0
2,66.008,-1799.147110,-1686.444593,-112.702517,339.158,-112.702517,-9.461727,-0.000041,-2.607738e-06,0.999997,1.0,39503.957611,0.172838,0.014519,3.497848,0.030262,0.0
3,66.049,-1797.251830,-1685.484681,-111.767149,339.199,-111.767149,-14.044180,-0.000061,-5.126419e-06,0.999995,1.0,39508.540064,0.172816,0.021605,3.490763,0.030478,0.0
4,66.091,-1787.505445,-1684.501357,-103.004088,339.241,-103.004088,-18.370351,-0.000080,-8.500475e-06,0.999991,1.0,39512.866235,0.172792,0.028862,3.483506,0.030700,0.0


La terza colonna (o 1° calcolata) rappresenta l'entalpia cumulativa:

$$
h\Delta H_i = \int Cp \, dT \approx \sum_i Cp_i \cdot \Delta T_i
$$

La quarta divide la terza colonna per $RT^2$, prendendo i valori di T dalla colonna 1 (K).
$$
\frac{h\Delta H_i}{R T^2}
$$

La colonna 5 integra la precedente (4). I limiti di integrazione sono calcolati dal primo valore ottenuto mentre Il limite superiore è il punto in cui la curva Cp(T) rientra nella tendenza lineare superiore, stimata con un extrapolazione (praticamente fino alla fine dei dati perché qui ho solo la transizione).
$$
\int \frac{h\Delta H_i}{R T^2} \, dT \approx \sum_i \frac{h\Delta H_i}{R T_i^2} \cdot \Delta T_i
$$

Col 6, funzione di partizione:
$$
Q = \exp\left( \int \frac{h\Delta H_i}{R T^2} \, dT \right)
$$

La colonna 7 rappresenta la frazione degli stati nativi $F_0$, definita:
$$
F_0 = \frac{1}{Q}
$$

Colonna 8 è l'entalpia cumulativa dalle alte fino alle basse temperature:
$$
\Delta H_n - h\Delta H_i
$$

9:
$$
Colonna9=Colonna8/RT^2
$$

Nella colonna 10 si calcola l'integrale con gli stessi limiti di 5:
$$
\int \frac{\Delta H_n - h\Delta H_i}{R T^2} \, dT \approx \sum_i \frac{\Delta H_n - h\Delta H_i}{R T_i^2} \cdot \Delta T_i
$$

La colonna 11 rappresento lo scarto tra il massimo valore della colonna 10 e la colonna 10 stessa:
$$
\text{Max} - \int \frac{\Delta H_n - h\Delta H_i}{R T^2} \, dT
$$

12: Questa colonna genera Fn, la frazione unfolded nello stato finale dei singoli filamenti:
$$
F_n = \exp\left( - \left[ \text{Max} - \int \frac{\Delta H_n - h\Delta H_i}{R T^2} \, dT \right] \right)
$$

L'ultima colonna (13) rappresenta la frazione di stati intermedi che altro non è che:
$$
F_i = 1 - F_0 - F_n
$$



## Studio degli errori:

Per ciascun campione è stato integrato il calore specifico corretto, usando:

$$
\Delta H = \int_{T_{min}}^{T_{max}} C_p(T) \, dT
$$

La procedura è ripetuta per 4 diversi modelli di baseline: lineare, cubica, a gradino e sigmoide.

Per stimare l'incertezza legata alla scelta del modello di baseline, viene calcolata la **deviazione standard** dei valori di ΔH ottenuti con le diverse baseline, assumendo che il contributo maggiore all'errore sia proprio legato a questa scelta.


In [4]:
records = []  # (Campione, Baseline, DeltaH)
for name, df in ci_finali_data.items():
    campione, baseline = name.split("_", 1)

    # Trova colonna con Cp finale
    col_cp = [c for c in df.columns if c.startswith("Cp_finale")][0]
    T = df["Temperatura"].values
    Cp = df[col_cp].values

    delta_H = np.trapezoid(Cp, T)
    records.append((campione, baseline, delta_H))

df_DH = pd.DataFrame(records, columns=["Campione", "Baseline", "DeltaH"])

# === 3. Calcolo errore tra baseline ===========
df_std = df_DH.groupby("Campione")["DeltaH"].std().rename("Errore_baseline")
df_DH = df_DH.merge(df_std, on="Campione").sort_values(["Campione", "Baseline"])

# === 4. Output ================================
pd.set_option("display.precision", 6)
print(df_DH.to_string(index=False))

Campione Baseline       DeltaH  Errore_baseline
      c3   cubica 30698.071911      5453.359082
      c3  gradino 42785.014249      5453.359082
      c3  lineare 39495.043091      5453.359082
      c3 sigmoide 41509.186818      5453.359082
      c5   cubica 16689.415547      7737.144316
      c5  gradino 33477.870112      7737.144316
      c5  lineare 29778.685419      7737.144316
      c5 sigmoide 32301.796159      7737.144316
      c7   cubica 18415.804950      4741.968061
      c7  gradino 28706.796858      4741.968061
      c7  lineare 26393.747000      4741.968061
      c7 sigmoide 27999.830893      4741.968061


## Valutazione della cooperatività della transizione termica

Per determinare se il processo di unfolding segue un modello a due stati cooperativo oppure coinvolge stati intermedi, si confrontano due entalpie:

- **ΔH<sub>cal</sub>**: entalpia calorimetrica, ottenuta integrando la curva di capacità termica:
  
  $$
  \Delta H_{cal} = \int C_p \, dT
  $$

- **ΔH<sub>vH</sub>**: entalpia di van’t Hoff, stimata assumendo una transizione a due stati:
  
  $$
  \Delta H_{vH} = \frac{4RT_m^2 \cdot C_p(T_m)}{\Delta H_{cal}}
  $$

dove:
- $ R $ è la costante dei gas (1.987 cal/mol·K),
- $ T_m $ è la temperatura al picco massimo della curva $ C_p $,
- $ C_p(T_m) $ è il valore di capacità termica al picco.

### Interpretazione del rapporto:

Si calcola:

$$
\frac{\Delta H_{vH}}{\Delta H_{cal}}
$$

E si valuta secondo le soglie seguenti:

- **> 0.95** → Transizione altamente cooperativa (modello a due stati)
- **0.85 – 0.95** → Transizione con deviazioni moderate dal modello a due stati
- **< 0.85** → Transizione non cooperativa, con presenza di più stati intermedi

Questo criterio aiuta a stabilire se è necessario applicare un'analisi più complessa (es. deconvoluzione Freire–Biltonen) o se un semplice modello a due stati è sufficiente.


In [5]:
def interpreta_cooperativita(ratio):
    if np.isnan(ratio):
        return "Valore non disponibile"
    elif ratio < 0.85:
        return "Transizione non cooperativa"
    elif ratio < 0.95:
        return "Moderata deviazione dal modello a due stati"
    else:
        return "Alta cooperatività (modello a due stati)"


### Stima della temperatura di melting $ T_m $ mediante fit analitico

Per ottenere una stima accurata della temperatura di melting $ T_m $, il tratto di picco del segnale $ C_p(T) $ è stato fittato utilizzando una forma semplificata della **funzione di Van’t Hoff**, particolarmente adatta a descrivere transizioni cooperative tra due stati.

La funzione utilizzata per il fit è:

$$
C_p(T) = \frac{A}{\cosh^2\left( \frac{T - T_m}{b} \right)}
$$

dove:
- $ A $ è un parametro proporzionale all’altezza massima del picco,
- $ T_m $ è la temperatura di melting (valore centrale della transizione),
- $ b $ è un parametro legato alla larghezza della transizione.

Questa funzione deriva da una formulazione del calore specifico per una transizione a due stati ed è in grado di catturare la forma tipica dei picchi di unfolding proteico o nucleico.

Il fit è stato effettuato per ciascun campione, e la stima di $ T_m $ ottenuta è stata poi utilizzata nel calcolo dell’entalpia di Van’t Hoff.


In [6]:
from scipy.optimize import curve_fit
import ipywidgets as widgets
from IPython.display import display
import numpy as np

# Funzione di van’t Hoff
def vanthoff_model(T, A, Tm, b):
    return A / np.cosh((T - Tm) / b)**2

# Dizionario per Tm stimati
Tm_dict = {}         # key = "c3_lineare", value = Tm in Kelvin
fit_param_dict = {}  # key = "c3_lineare", value = (A, Tm, b)
fit_sigma_dict = {}  

# === Calcolo batch dei Tm stimati ===
for name, df in ci_finali_data.items():

    T = df["Temperatura"].values
    Cp = df[[c for c in df.columns if c.startswith("Cp_finale")][0]].values

    A0, Tm0, b0 = Cp.max(), T[np.argmax(Cp)], 2.0
    popt, pcov = curve_fit(vanthoff_model, T, Cp, p0=[A0, Tm0, b0])
    A_fit, Tm_fit, b_fit = popt
    sigma_Tm = np.sqrt(pcov[1, 1]) if pcov is not None else np.nan   # errore 1 σ

    fit_param_dict[name] = (A_fit, Tm_fit, b_fit)
    fit_sigma_dict[name] = sigma_Tm           #  <<--- salva l’errore
    Tm_dict[name] = Tm_fit

# Estrai nomi validi
campioni = sorted(set(name.split("_")[0] for name in Tm_dict.keys()))
baselines = sorted(set(name.split("_")[1] for name in Tm_dict.keys()))

# Funzione interattiva (usa solo i fit salvati)
def plot_fit_interattivo(campione, baseline):
    key = f"{campione}_{baseline}"

    if key not in ci_finali_data or key not in fit_param_dict:
        print(f"Fit non disponibile per {key}")
        return

    df = ci_finali_data[key]
    T = df["Temperatura"].values
    Cp = df[[c for c in df.columns if c.startswith("Cp_finale")][0]].values

    A_fit, Tm_fit, b_fit = fit_param_dict[key]
    T_fit = np.linspace(T.min(), T.max(), 500)
    Cp_fit = vanthoff_model(T_fit, A_fit, Tm_fit, b_fit)

    plt.figure(figsize=(8, 5))
    plt.plot(T, Cp, 'o', label="Dati")
    plt.plot(T_fit, Cp_fit, '-', label=f"Fit van’t Hoff\nTm ≈ {Tm_fit:.2f} °C")
    plt.xlabel("Temperatura (°C)")
    plt.ylabel("Cp")
    plt.title(f"Fit van’t Hoff – {key}")
    plt.legend()
    plt.grid(True)
    plt.show()

# Widget interattivi
campione_widget = widgets.Dropdown(options=campioni, description="Campione")
baseline_widget = widgets.Dropdown(options=baselines, description="Baseline")

ui = widgets.HBox([campione_widget, baseline_widget])
out = widgets.interactive_output(plot_fit_interattivo, {"campione": campione_widget, "baseline": baseline_widget})

display(ui, out)


Output()

In [7]:
# --- all’inizio, hai già salvato i parametri del fit ----------
# fit_param_dict[name] = (A_fit, Tm_fit, b_fit)

R = 1.987  # cal/mol·K
records = []

for _, row in df_DH.iterrows():
    campione  = row["Campione"]
    baseline  = row["Baseline"]
    H_cal     = row["DeltaH"]          # già in cal/mol
    err_base  = row["Errore_baseline"]
    name      = f"{campione}_{baseline}"

    if name not in fit_param_dict or np.isnan(H_cal):
        continue

    A_fit, Tm_fit_C, b_fit = fit_param_dict[name]
    Tm_K = Tm_fit_C + 273.15           # per la formula di van’t Hoff
    Cp_Tm = A_fit                      # **** qui ‼️ non più interpolazione

    # ΔH_vH sempre positivo se A_fit > 0
    deltaH_vH = (4 * R * Tm_K**2 * Cp_Tm) / H_cal

    ratio = deltaH_vH / H_cal
    interp = interpreta_cooperativita(ratio)

    records.append({
        "Campione": campione,
        "Baseline": baseline,
        "ΔH_cal (cal/mol)": H_cal,
        "Errore_baseline": err_base,
        "ΔH_vH (cal/mol)": deltaH_vH,
        "HvH / Hcal": ratio,
        "Cooperatività": interp
    })

df_cooperativita = (pd.DataFrame(records)
                    .sort_values(["Campione","Baseline"])
                    .reset_index(drop=True))

display(df_cooperativita.head(12))


,Campione,Baseline,ΔH_cal (cal/mol),Errore_baseline,ΔH_vH (cal/mol),HvH / Hcal,Cooperatività
0,c3,cubica,30698.071911,5453.359082,85909.884932,2.798543,Alta cooperatività (modello a due stati)
1,c3,gradino,42785.014249,5453.359082,77387.539778,1.808753,Alta cooperatività (modello a due stati)
2,c3,lineare,39495.043091,5453.359082,79975.731144,2.024956,Alta cooperatività (modello a due stati)
3,c3,sigmoide,41509.186818,5453.359082,78244.893475,1.885002,Alta cooperatività (modello a due stati)
4,c5,cubica,16689.415547,7737.144316,101323.859175,6.071145,Alta cooperatività (modello a due stati)
5,c5,gradino,33477.870112,7737.144316,86863.077409,2.594642,Alta cooperatività (modello a due stati)
6,c5,lineare,29778.685419,7737.144316,90760.805154,3.047845,Alta cooperatività (modello a due stati)
7,c5,sigmoide,32301.796159,7737.144316,87627.034927,2.712760,Alta cooperatività (modello a due stati)
8,c7,cubica,18415.804950,4741.968061,96441.174520,5.236870,Alta cooperatività (modello a due stati)
9,c7,gradino,28706.796858,4741.968061,88861.379941,3.095482,Alta cooperatività (modello a due stati)


## Spink method analysis:

### Trattamento numerico delle frazioni molecolari (F₀, Fᵢ, Fₙ)

Nel modello di deconvoluzione termodinamica basato sulle equazioni di Freire–Biltonen, le frazioni molecolari calcolate $ F_0 $, $ F_i $ e $ F_n $ dovrebbero teoricamente rispettare i seguenti vincoli fisici:

- $ 0 \leq F_0, F_i, F_n \leq 1 $
- $ F_0 + F_i + F_n = 1 $ (entro la precisione numerica)

Tuttavia, nella pratica, i dati sperimentali (es. Cp rumorosi, correzione di baseline imperfetta) e l'integrazione numerica discreta possono introdurre errori che causano:

- Valori negativi o superiori a 1 per alcune frazioni
- Valori di $ F_i $ negativi a causa di propagazione dell’errore

Per questo motivo, in questa implementazione, **è stata applicata la funzione `np.clip` per forzare le frazioni nel range [0, 1]**, garantendo:

- **Consistenza fisica** dei risultati (nessuna probabilità negativa o >1)
- **Stabilità numerica** rispetto a picchi, disallineamenti di baseline o piccoli errori di quadratura
- **Robustezza su dati reali e non idealizzati**

> **Nota bene**:  questa correzione numerica è applicata solo a valle del calcolo delle frazioni, e non modifica né la funzione di partizione Q né i parametri termodinamici. Si tratta di una misura conservativa per evitare artefatti numerici nei risultati interpretativi.

> **Nota sulla compatibilità con Origin®**: nei riferimenti disponibili (Spink, 2008), il comportamento interno di Origin® non è documentato in dettaglio. Non è possibile verificare se applichi clipping esplicito alle frazioni, ma si presume che vengano adottate misure analoghe per garantire la validità fisica dei risultati, anche se non rese pubbliche.


In [ ]:
def plot_frazioni_interattivo(df_spink_dict):
    """
    Visualizza F0, Fi, Fn in funzione della temperatura per ogni campione, interattivamente.
    """
    opzioni = sorted(df_spink_dict.keys())

    @interact(dataset=Dropdown(options=opzioni, description='Campione:'))
    def _plot(dataset):
        df = df_spink_dict[dataset]
        T = df['Temperatura']
        
        plt.figure(figsize=(10, 6))
        plt.plot(T, df['Fn'], label='Fn (unfolded)', color='red')
        plt.plot(T, df['Fi'], label='Fi (intermediate)', color='green')
        plt.plot(T, df['F0'], label='F0 (native)', color='orange')
        
        plt.title(f'Fn, Fi, F0 vs Temperatura - {dataset}')
        plt.xlabel('Temperatura (°C)')
        plt.ylabel('Frazione molecolare')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()


plot_frazioni_interattivo(df_spink)

interactive(children=(Dropdown(description='Campione:', options=('c3_cubica', 'c3_gradino', 'c3_lineare', 'c3_…